# Table S09. Overlapping susceptibility-associated intervals across

toxicant condition pairs

In [ ]:

library(data.table)
library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'kableExtra'

The following objects are masked from 'package:flextable':

    as_image, footnote

The following object is masked from 'package:dplyr':

    group_rows

here() starts at /Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript

$strain_table
$strain_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.html"

$strain_table$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.tsv.zip"

$strain_table$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.docx"

$strain_table$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.csv"


$tox_table_ft
$tox_table_ft$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.html"

$tox_table_ft$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.tsv.zip"

$tox_table_ft$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.docx"

$tox_table_ft$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.csv"


$trait_table
$trait_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S3/table_S3.html"

$trait_table$tsv
[1] "

# Inputs

In [ ]:
# toxicant condition metadata
tox_meta_fn <- "data/processed/tox_data/tox_metadata.csv"

# LD between peak markers from plink
inbred_qtl_overlap_peak_ld_fn <- "data/processed/interval_overlap/peak_ld_rsq.csv"

inbred_qtl_overlaps_fn <- "data/processed/interval_overlap/inbred_qtl_overlaps.csv"

# interval overlap summary output by `find_overlaps.R`
inbred_qtl_overlaps_summary_fn <- "data/processed/interval_overlap/inbred_qtl_overlaps_summary.csv"


# Main

Join the interval overlaps with LD between peak markers

In [ ]:
# load the toxicant condition metadata
tox_meta <- fread(tox_meta_fn)

# load the inbred qtl overlaps
inbred_qtl_overlaps <- data.table::fread(inbred_qtl_overlaps_fn)

# load the interval overlap summary
inbred_qtl_overlaps_summary <- data.table::fread(inbred_qtl_overlaps_summary_fn)

# load the LD between peak markers
inbred_qtl_overlap_peak_ld <- 
  data.table::fread(
    inbred_qtl_overlap_peak_ld_fn, 
    col.names = c( "peakidA", "peakidB", "R2")
  )%>%
  # format peakids to match those in the overlaps data instead of plink required format
  dplyr::mutate(
    peakidA = stringr::str_replace(peakidA, pattern = "_", replacement = ":"),
    peakidB = stringr::str_replace(peakidB, pattern = "_", replacement = ":") 
  )

# Create mapping from trait to nice_drug_label2
trait_to_nice_label <- setNames(tox_meta$nice_drug_label2, tox_meta$trait)

### Join the overlaps to the LD data ###
inbred_qtl_overlap_peak_w_ld <- inbred_qtl_overlaps_summary %>%
  # update the traitA and traitB columns to use the nice_drug_label2
  dplyr::mutate(
    traitA = trait_to_nice_label[traitA],
    traitB = trait_to_nice_label[traitB]
  ) %>%
  dplyr::left_join(inbred_qtl_overlap_peak_ld, by = c("peakidA", "peakidB"))


Create formatted flextable of the overlaps with LD

In [ ]:

# restrict to the columns we want
overlap_pre <- inbred_qtl_overlap_peak_w_ld %>%
  dplyr::select(
    traitA,
    traitB,
    peakidA,
    peakidB,
    R2
  )

# create a column to combine traitA and traitB
overlap_pre <- overlap_pre %>%
  mutate(
    trait_pair = paste(traitA, traitB, sep = " & "),
    interval_pair = paste(peakidA, peakidB, sep = " & ")
  )

# create a grouped flextable
overlap_pre_group <- overlap_pre %>%
  dplyr::arrange(trait_pair, R2) %>%
  as_grouped_data(
    groups = c("trait_pair")
  )

# create a flextable
overlap_ft <- overlap_pre_group %>%
  flextable::as_flextable(
    col_keys = c("trait_pair", "interval_pair", "R2"),
    hide_grouplabel = TRUE
  ) %>%
  flextable::bold(part = "header") %>%
  # when R2 is NA make the row text bold
  flextable::bold(j = 1, i = ~ !is.na(trait_pair), bold = TRUE, part = "body") %>%
  flextable::set_header_labels(
    trait_pair = "Trait Pair",
    interval_pair = "Interval Pair",
    R2 = "LD (r²)"
    ) %>%
  flextable::set_table_properties(layout = "autofit") %>%
  flextable::align(align = "left", part = "all") %>%
  flextable::set_table_properties(
    layout = "autofit",
    width = 1,
    opts_word = list(
      split = FALSE,
      keep_with_next = TRUE
    )
  ) %>%
  flextable::paginate(
    group = "trait_pair",
    group_def = c("rle")
  )

overlap_ft


Interval Pair LD (r²) 2,4-D & TPhP 6.25 uM V:14453636 & V:7623783 0.00 Atrazine & Chlorothalonil II:12658388 & II:5392976 0.01 Carboxin & Arsenic III:2527992 & III:1642838 0.12 Carboxin & Chlorothalonil III:2527992 & III:2605810 0.40 Chlorothalonil & Arsenic III:2605810 & III:1642838 0.19 III:1455164 & III:1642838 0.35 II:13921653 & II:13692527 0.38 Chlorothalonil & Nickel II:5392976 & II:12203935 0.03 Mancozeb & Chlorothalonil II:10305644 & II:5392976 0.03 Mercury & Arsenic III:2461452 & III:1642838 0.00 Mercury & Carbaryl I:2994073 & I:2596756 0.00 Mercury & Carboxin III:2461452 & III:2527992 0.02 Mercury & Chlorothalonil III:2461452 & III:2605810 0.01 III:3716201 & III:4478684 0.06 Mercury & Paraquat 62.5 uM I:2994073 & I:3282224 0.00 Mercury & Silver 7.8 uM II:2730209 & II:3016298 0.01 Mercury & TPhP 6.25 uM V:15476283 & V:7623783 0.07 Paraquat 62.5 uM & Arsenic II:13675568 & II:13692527 0.03 Paraquat 62.5 uM & Chlorothalonil II:13675568 & II:13921653 0.05 Paraquat 62.5 uM & Zinc I:11258815 & I:4884561 0.01 Silver 7.8 uM & Chlorothalonil II:11883855 & II:5392976 0.00 I:1622740 & I:1741155 0.28

In [ ]:
# prepare the CSV output with nice column names
overlap_csv <- overlap_pre %>%
  dplyr::select(trait_pair, interval_pair, R2) %>%
  dplyr::rename(
    "Trait Pair" = trait_pair,
    "Interval Pair" = interval_pair,
    "LD (r²)" = R2
  )

# save the flextable using standardized naming
save_supp_table_flextable(overlap_ft, "overlap_ft")
save_supp_table_csv(overlap_csv, "overlap_ft")
